---
id: "DE-01"
title: "Batch ingestion from WMS CSV to Data Warehouse"
specialty: "Data Engineering"
process: "Deliver"
level: "Intermediate"
tags: ["etl", "warehouse", "inventory", "python", "sql"]
estimated_time_min: 45
---

## 🎯 Contexto del Notebook

### ¿Qué?
Ingesta batch de datos de inventario desde archivos CSV (simulando WMS) hacia un modelo analítico.

### ¿Por qué?
Los sistemas WMS generan archivos planos diarios. Consolidarlos en un warehouse permite análisis histórico, tendencias y alertas.

### ¿Para qué?
- Construir tablas de hechos de inventario
- Habilitar reportes de cobertura, rotación y obsolescencia
- Base para modelos de optimización de stock

### ¿Cuándo?
Ejecutar diariamente en horarios de baja operación (ej: 2 AM) post-cierre de turno WMS.

### ¿Cómo?
1. Leer CSV desde `data/raw/inventory.csv`
2. Validar esquema y tipos
3. Transformar (agregar timestamp, calcular métricas)
4. Escribir a `data/processed/` o conectar a DB

In [ ]:
import pandas as pd
from pathlib import Path
from src.utils.paths import DATA_RAW, DATA_PROCESSED, ensure_dirs
from src.utils.logging import get_logger

logger = get_logger('DE-01')
ensure_dirs()
logger.info('Iniciando ingesta batch de inventario')

In [ ]:
# Leer CSV de inventario
df = pd.read_csv(DATA_RAW / 'inventory.csv')
logger.info(f'Registros cargados: {len(df)}')
print(df.head())
print(df.info())

In [ ]:
# Transformaciones básicas
df['ingestion_date'] = pd.Timestamp.now()
df['on_hand_value'] = df['on_hand'] * 10  # mock: precio unitario
print(df[['location_id','sku','on_hand','on_hand_value']].head())

In [ ]:
# Escribir a processed
output = DATA_PROCESSED / 'inventory_fact.parquet'
df.to_parquet(output, index=False)
logger.info(f'Datos escritos en {output}')
print('✅ Ingesta completada')